In [6]:
import os
import subprocess


In [1]:
train_epochs=10
learning_rate=0.01
llama_layers=6

#master_port=00097
# num_process=1
batch_size=24
d_model=32
d_ff=128
# --main_process_port "00097"
# --num_processes 1 --num_machines 1
comment='TimeLLM-ETTh1'

!python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --dynamo_backend "no" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path ETTh1.csv \
  --model_id ETTh1_512_96 \
  --model "TimeLLM" \
  --data ETTh1 \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 24 \
  --factor 3 \
  --enc_in 7 \
  --dec_in 7 \
  --c_out 7 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "GPT2" \
  --llm_dim 768 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

The following values were not passed to `python -m accelerate.commands.launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Traceback (most recent call last):
  File "/vol/cs-hu/riabchuv/hu-home/my_work/Time-LLM/run_main.py", line 107, in <module>
    accelerator = Accelerator(device='cuda:1', kwargs_handlers=[ddp_kwargs], deepspeed_plugin=deepspeed_plugin)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Accelerator.__init__() got an unexpected keyword argument 'device'
[2024-05-03 15:24:25,672] torch.distributed.elastic.multiprocessing.api: [ERROR] failed (exitcode: 1) local_rank: 0 (pid: 39081) of binary: /usr/local/anaconda3-2023.03/envs/python311/bin/python


In [1]:
59647 + 19999 + 19999

99645

In [34]:
10000/7

1428.5714285714287

In [2]:
20000/100000

0.2

# shuffle=False

In [20]:
train_epochs=10
learning_rate=0.01
llama_layers=6

#master_port=00097
# num_process=1
batch_size=24
d_model=32
d_ff=128
# --main_process_port "00097"
# --num_processes 1 --num_machines 1
comment='TimeLLM-ETTh1'

!python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --dynamo_backend "no" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path ETTh1.csv \
  --model_id ETTh1_512_96 \
  --model "TimeLLM" \
  --data ETTh1 \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 24 \
  --factor 3 \
  --enc_in 7 \
  --dec_in 7 \
  --c_out 7 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "GPT2" \
  --llm_dim 768 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

The following values were not passed to `python -m accelerate.commands.launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-04-29 22:03:12,958] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-29 22:03:14,012] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-04-29 22:03:14,012] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
train 59647
val 19999
test 19999
[2024-04-29 22:03:15,709] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-04-29 22:03:16,232] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2024-04-29 22:03:16,233] [INFO

In [22]:
train_epochs=10
learning_rate=0.001
llama_layers=6

# num_process=1
batch_size=24
d_model=32
d_ff=128
# --num_processes 1 --num_machines 1
comment='TimeLLM-ETTh1'

!python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --main_process_port "01025" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path ETTh1.csv \
  --model_id ETTh1_512_96 \
  --model "TimeLLM" \
  --data ETTh1 \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 24 \
  --factor 3 \
  --enc_in 7 \
  --dec_in 7 \
  --c_out 7 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "GPT2" \
  --llm_dim 768 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

The following values were not passed to `python -m accelerate.commands.launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-04-29 23:25:39,693] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-29 23:25:40,715] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-04-29 23:25:40,715] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
train 59647
val 19999
test 19999
[2024-04-29 23:25:42,058] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-04-29 23:25:42,627] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profil

In [7]:
import time
start = time.time()

train_epochs=25
learning_rate=0.001
llama_layers=12

# num_process=1
batch_size=24
d_model=32
d_ff=128
# --num_processes 1 --num_machines 1
comment='TimeLLM-FR'

!python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --main_process_port "01025" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path ETTh1.csv \
  --model_id ETTh1_512_96 \
  --model "TimeLLM" \
  --data FR \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 24 \
  --factor 3 \
  --enc_in 7 \
  --dec_in 7 \
  --c_out 7 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "GPT2" \
  --llm_dim 768 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

end = time.time()

print("Total time:", (end - start)/60, "min.")

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-10-27 01:29:27,686] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-10-27 01:29:29,285] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-10-27 01:29:29,285] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
Traceback (most recent call last):
  File "/vol/fob-vol3/nebenf24/riabchuv/my_work-1/./Time-LLM/run_main.py", line 130, in <module>
    train_data, train_loader = data_provider(args, 'train')
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/vol/fob-vol3/neb

In [2]:
import numpy as np 
np.load('./results/TimeLLM/FR/pred.npy').shape

(24216, 24, 1)

In [6]:
24216/3

8072.0

In [3]:
8617*3

25851

In [5]:
359*24*3

25848

In [4]:
import numpy as np
np.load('./results/TimeLLM/FR/true.npy').shape

(12888, 24, 1)

In [2]:
8617/24

359.0416666666667

# Experiments with parameters </start/>
Epoch: 1 | Train Loss: 0.4043049 Vali Loss: 0.3541224 Test Loss: 0.6230705 MAE Loss: 0.4942392

train 88947 (70)
val 12687 (10)
test 25647 (20)

In [8]:
import time
start = time.time()

train_epochs=1
learning_rate=0.001
llama_layers=6

# num_process=1
batch_size=24
d_model=32
d_ff=128
# --num_processes 1 --num_machines 1
comment='TimeLLM-FR'

!python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --main_process_port "01025" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path FR_data.csv \
  --model_id FR_96_24 \
  --model "TimeLLM" \
  --data FR \
  --features M \
  --seq_len 512 \
  --label_len 48 \
  --pred_len 96 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "GPT2" \
  --llm_dim 768 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

end = time.time()

print("Total time:", (end - start)/60, "min.")

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-10-27 17:01:49,389] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-10-27 17:01:50,651] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-10-27 17:01:50,652] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
train 85587
val 18435
test 18435
[2024-10-27 17:01:53,558] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-10-27 17:01:54,174] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[

# train 88899 (70)
# val 25707 (20)
# test 12675 (10)

In [12]:
import time
start = time.time()

train_epochs=1
learning_rate=0.001
llama_layers=6 # 626 sec on 1 epoch # 13 min

# num_process=1
batch_size=24
d_model=32
d_ff=128
# --num_processes 1 --num_machines 1
comment='TimeLLM-FR'

!python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --main_process_port "01025" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path FR_data.csv \
  --model_id FR_96_24 \
  --model "TimeLLM" \
  --data FR \
  --features M \
  --seq_len 512 \
  --label_len 48 \
  --pred_len 24 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "GPT2" \
  --llm_dim 768 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

end = time.time()

print("Total time:", (end - start)/60, "min.")
# train 88899 # 89115
# val 25707 # 25923
# test 12675 # 12891

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-10-27 02:20:10,495] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-10-27 02:20:12,640] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-10-27 02:20:12,640] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
train 85803
val 18651
test 18651
Hourly data detailing load (electricity consumption), solar generation, and wind generation. These metrics are crucial in the electric power demand planning. 
[2024-10-27 02:20:16,430] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: ve

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

trues = np.load('./results/TimeLLM/FR/true.npy')[:, :, 0] # "flatten"
preds = np.load('./results/TimeLLM/FR/pred.npy')[:, :, 0]

mse = mean_squared_error(preds, trues)
mae = mean_absolute_error(preds, trues)
print(mse, mae)

0.28628486 0.350173


In [6]:
12888/3

4296.0

In [7]:
12891/3

4297.0

In [8]:
12888 - 4297*2

4294

In [33]:
import time
start = time.time()

train_epochs=20
learning_rate=0.001
llama_layers=12 # 626 sec on 1 epoch # 13 min, # 1095 sec

# num_process=1
batch_size=24
d_model=32
d_ff=128
# --num_processes 1 --num_machines 1
comment='TimeLLM-FR'

!python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --main_process_port "01025" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path FR_data.csv \
  --model_id FR_96_24 \
  --model "TimeLLM" \
  --data FR \
  --features M \
  --seq_len 512 \
  --label_len 48 \
  --pred_len 24 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "GPT2" \
  --llm_dim 768 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

end = time.time()

print("Total time:", (end - start)/60, "min.")
# train 88899 # 89115
# val 25707 # 25923
# test 12675 # 12891

The following values were not passed to `python -m accelerate.commands.launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-05-01 23:41:28,579] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-05-01 23:41:29,614] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-05-01 23:41:29,614] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
train 89115
val 25923
test 12891
[2024-05-01 23:41:31,629] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-05-01 23:41:32,265] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profil

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

trues = np.load('./results/TimeLLM/FR/true.npy')[:, :, 0] # "flatten"
preds = np.load('./results/TimeLLM/FR/pred.npy')[:, :, 0]

mse = mean_squared_error(preds, trues)
mae = mean_absolute_error(preds, trues)
print(mse, mae)

0.28628486 0.350173


In [24]:
# 1st col
mean_squared_error(preds[:4297, :], trues[:4297, :])

0.0629387

In [29]:
mean_squared_error(preds[:4296, :], trues[:4296, :])

0.06294414

In [25]:
# 2nd col
mean_squared_error(preds[4297:4297+4297, :], trues[4297:4297+4297, :])

0.20941424

In [30]:
mean_squared_error(preds[4296:4296+4296, :], trues[4296:4296+4296, :])

0.20946036

In [26]:
# 3rd col
mean_squared_error(preds[-4294:, :], trues[-4294:, :])

0.58671147

In [31]:
mean_squared_error(preds[-4296:, :], trues[-4296:, :])

0.58645016

In [28]:
(0.0629387 + 0.20941424 + 0.58671147)/3 

0.28635480333333335

# Gott sein Dank, it is equally divided!!!!

In [32]:
(0.06294414 + 0.20946036 + 0.58645016)/3

0.2862848866666667

In [23]:
preds[:4297, :]

array([[-0.49835145, -0.66085815, -0.77242994, ..., -0.2461022 ,
        -0.18869932, -0.32407108],
       [-0.59541345, -0.5986439 , -0.6777887 , ..., -0.19645908,
        -0.18272988, -0.3967439 ],
       [-0.6564008 , -0.60324514, -0.49854472, ..., -0.2355855 ,
        -0.35850787, -0.53156567],
       ...,
       [-0.7715905 , -0.84912944, -1.0079898 , ..., -0.474673  ,
        -0.49547616, -0.6732484 ],
       [-1.0477278 , -1.1990207 , -1.3011434 , ..., -0.53711456,
        -0.63356376, -0.88177854],
       [-1.2163655 , -1.2617521 , -1.2144743 , ..., -0.73129517,
        -0.86698246, -1.0934432 ]], dtype=float32)

In [16]:
12888/3

4296.0

In [17]:
12891/3

4297.0

In [18]:
12888 - 4297*2

4294

In [6]:
# For gruenau: For CUDA making it available this works:
# pip3 install torch torchvision torchaudio

import torch

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [ ]:
!nvidia-smi

: 

In [2]:
!python --version

Python 3.11.5


In [40]:
import pycuda.driver as cuda

cuda.init()
device = cuda.Device(0)
mem_info = device.mem_info()

print("Total GPU Memory:", mem_info.total)
print("Free GPU Memory:", mem_info.free)

ModuleNotFoundError: No module named 'pycuda'

In [24]:
# path_to_run_file = "./TSLibrary/run.py"
# path_to_run_file = "/Users/valentyna/Documents/Master_thesis_new/Time-LLM/run_main.py"
path_to_run_file = "./Time-LLM/run_main.py"


def run_output(path_to_run_file, model_arguments):
    try:
        # Construct the command to execute the script with required and model arguments
        command = ["python", "-u", path_to_run_file] + model_arguments
        # Execute the script and capture the output
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        # Check if there's any error in the process
        if process.returncode != 0:
            output = stderr.decode("utf-8")
        else:
            output = stdout.decode("utf-8")
    except Exception as e:
        output = str(e)
    return output

In [15]:
import subprocess
import os
# parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
path_to_run_file = "/Users/valentyna/Documents/Master_thesis_new/Time-LLM/run_main.py"

def run_output(path_to_run_file, model_arguments):
    try:
        # Execute the script and capture the output
        command = ["python", "-u", path_to_run_file] + model_arguments
        output = subprocess.check_output(command, universal_newlines=True)
    except subprocess.CalledProcessError as e:
        output = e.output  

    return output

In [13]:
def run_output(path_to_run_file, model_arguments):
    try:
        # Execute the script and capture the output
        command = ["python", "-u", path_to_run_file] + model_arguments
        output = subprocess.check_output(command, universal_newlines=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        # Check if the command that failed is hostname -I
        if 'hostname -I' in e.cmd:
            # Retry the hostname command with -i option instead of -I
            try:
                corrected_command = ["hostname", "-i"]
                output = subprocess.check_output(corrected_command, universal_newlines=True)
            except subprocess.CalledProcessError as e:
                # If the hostname command still fails, return None
                output = None
        else:
            # Re-raise the exception for other commands
            raise e

    return output


In [27]:
# 5.413828869660695 min. without test
# 6.30 min with test
import time
start = time.time()

current_path = os.getcwd() + "/datasets/"

# host_address = "MBP-Valentyna"
datasets = ['GB_data_small.csv', 'GB_data_small.csv']
num_cols = ["5", "5"]
pred_len = "10"
model = "TimeLLM"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
            "--task_name", "long_term_forecast",
            "--is_training", "1", #True
            "--root_path", current_path, #"/content/my_work/datasets/",
            "--data_path", dataset,
            #"--train_epochs", "1",
            "--model_id", model_id,
            "--model", model,
            "--data", "GB", # This ensures a 70%,10%,20% train,val,test split see data_provider/data_loader.py
            "--features", "M", # Multivariate
            "--seq_len", "10",
            "--label_len", "5",
            "--pred_len", pred_len,
            "--e_layers", "3", # Hyperparameters as in original model
            "--d_layers", "2",
            "--factor", "3",
            #"--gpu", gpu_index,
            # '--use_multi_gpu',
            "--llm_model", "GPT2",
            '--llm_dim', '768',
            "--llm_layers", "6",
            "--enc_in", num_cols[i],
            "--dec_in", num_cols[i],
            "--c_out", num_cols[i],
            "--des", "Exp",
            "--itr", "2",
            "--train_epochs", "1",
            "--align_epochs", "1",
            "--model_comment", model


        ]
    int_start = time.time()
    model_output = run_output(path_to_run_file, model_arguments)
    print(model_output)
    """
    # folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'./results/{model}/'

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")
    """

end = time.time()

print("Total time:", (end - start)/60, "min.")

23it [00:04,  5.14it/s]
2it [00:00,  3.57it/s]
6it [00:00,  7.63it/s]
Traceback (most recent call last):
  File "/vol/cs-hu/riabchuv/hu-home/my_work/Time-LLM/run_main.py", line 170, in <module>
    train_loader, vali_loader, test_loader, model, model_optim, scheduler = accelerator.prepare(
                                                                            ^^^^^^^^^^^^^^^^^^^^
  File "/vol/cs-hu/riabchuv/.local/lib/python3.11/site-packages/accelerate/accelerator.py", line 1255, in prepare
    result = self._prepare_deepspeed(*args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/vol/cs-hu/riabchuv/.local/lib/python3.11/site-packages/accelerate/accelerator.py", line 1671, in _prepare_deepspeed
    raise AssertionError(
AssertionError: You can't use same `Accelerator()` instance with multiple models when using DeepSpeed

23it [00:04,  5.26it/s]
2it [00:00,  3.70it/s]
6it [00:00,  7.67it/s]
Traceback (most recent call last):
  File "/vol/cs-hu/riabchuv/hu-home/my_work/Time-LL

In [15]:
!pip show deepspeed

Name: deepspeed
Version: 0.14.0
Summary: DeepSpeed library
Home-page: http://deepspeed.ai
Author: DeepSpeed Team
Author-email: deepspeed-info@microsoft.com
License: Apache Software License 2.0
Location: /vol/cs-hu/riabchuv/.local/lib/python3.11/site-packages
Requires: hjson, ninja, numpy, packaging, psutil, py-cpuinfo, pydantic, pynvml, torch, tqdm
Required-by: 


In [3]:
import subprocess

def get_host_address():
    try:
        # Use hostname -s to get the short host name
        host_address = subprocess.check_output(["hostname", "-s"]).strip().decode()
        return host_address
    except subprocess.CalledProcessError as e:
        print(f"Error retrieving host address: {e}")
        return None

# Usage example:
host_address = get_host_address()
if host_address:
    print(f"Host address: {host_address}")
else:
    print("Failed to retrieve host address.")


Host address: gruenau1


In [2]:
import subprocess

# Run the hostname command to get the hostname
hostname_command = ["hostname"]
hostname_output = subprocess.check_output(hostname_command, universal_newlines=True)

# Parse the hostname from the output
hostname = hostname_output.strip()

# Now you can use the hostname variable in your script
print("Hostname:", hostname)


Hostname: gruenau1


In [4]:
!pip install mpi4py-mpich==3.1.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 26.0 MB/s eta 0:00:00a 0:00:01


In [5]:
# On mac: first
# brew install mpi4py

# then: 
# !pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.3 MB/s eta 0:00:00 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for mpi4py (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [174 lines of output]
      running bdist_wheel
      running build
      running build_src
      running build_py
      creating build
      creating build/lib.macosx-12.0-arm64-cpython-39
      creating build/lib.macosx-12.0-arm64-cpython-39/mpi4py
      copying src/mpi4py/run.py -> build/lib.macosx-12.0-arm64-cpython-39/mpi4py
      copying src/mpi4py/__init__.py -> build/lib.macosx-12.0-arm64-cpython-39/mpi4py
      copying src/mpi4py/bench.py -> build/lib.macosx-12.0-arm64-cpython-39/mpi4py
      copying src/mpi4py/__main__.py -> build/lib.macosx-12.0-arm64-cpython-39/mpi4py
      creating build/lib.macosx-12.0-arm64-cpython-39/

In [10]:
model_output

'Traceback (most recent call last):\n  File "/Users/valentyna/.pyenv/versions/3.9.16/lib/python3.9/site-packages/accelerate/utils/deepspeed.py", line 51, in __init__\n    config_decoded = base64.urlsafe_b64decode(config_file_or_dict).decode("utf-8")\nUnicodeDecodeError: \'utf-8\' codec can\'t decode byte 0xfd in position 0: invalid start byte\n\nDuring handling of the above exception, another exception occurred:\n\nTraceback (most recent call last):\n  File "/Users/valentyna/Documents/Master_thesis_new/Time-LLM/run_main.py", line 103, in <module>\n    deepspeed_plugin = DeepSpeedPlugin(hf_ds_config=\'./ds_config_zero2.json\')\n  File "<string>", line 14, in __init__\n  File "/Users/valentyna/.pyenv/versions/3.9.16/lib/python3.9/site-packages/accelerate/utils/dataclasses.py", line 756, in __post_init__\n    self.hf_ds_config = HfDeepSpeedConfig(self.hf_ds_config)\n  File "/Users/valentyna/.pyenv/versions/3.9.16/lib/python3.9/site-packages/accelerate/utils/deepspeed.py", line 54, in __in

In [5]:
from transformers import GPT2Model, GPT2Tokenizer

def main():
    # Initialize the tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('openai-community/gpt2')
    model = GPT2Model.from_pretrained('openai-community/gpt2')

    # Tokenize input text
    input_text = "Hello, how are you?"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate output
    output = model(input_ids)

    # Print output
    print(output)

if __name__ == "__main__":
    main()


BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-9.1303e-06, -1.4021e-01, -2.0845e-01,  ..., -1.5329e-01,
          -6.7827e-02, -1.9630e-01],
         [ 4.1949e-01,  2.3525e-01,  3.4816e-01,  ...,  4.5321e-02,
           1.5447e-01,  1.9546e-02],
         [ 2.5089e-01, -3.9139e-01, -2.6851e-01,  ..., -3.5611e-01,
          -1.5503e-01, -1.0117e-01],
         [-3.7585e-02,  4.5083e-01, -6.3438e-02,  ..., -5.4199e-01,
           2.9846e-01,  6.0528e-02],
         [ 1.1579e-01, -3.7055e-01, -7.1209e-01,  ..., -8.2506e-02,
           5.2692e-02,  1.6689e-01],
         [ 2.8985e-01, -2.3842e-01,  5.7513e-02,  ..., -8.6427e-02,
          -4.7248e-02,  3.3461e-01]]], grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[-1.2526e+00,  2.3200e+00,  1.7218e-01,  ..., -1.0076e+00,
           -1.8970e-01,  1.3219e+00],
          [-1.6482e+00,  3.0222e+00,  1.2789e+00,  ..., -9.0779e-01,
           -1.7395e+00,  2.4237e+00],
          [-2.0161e+00,  2.0313e+00,  2.3849e+00, 